In [1]:
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict
import numpy as np
import os
from PIL import Image
from sklearn.metrics import f1_score

In /home/holic/miniconda3/envs/tf1.11/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/holic/miniconda3/envs/tf1.11/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/holic/miniconda3/envs/tf1.11/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
def stitch_count(instr_path, file_path):    
    img_files = []
    if file_path:
        for f_path in file_path:
            with open(f_path, "r") as f:
                img_f = f.read().splitlines()
                img_files.extend(img_f)
    else:
        img_files = os.listdir(instr_path)

    element_counts = Counter()
    for f in img_files:
        if f.endswith('.png'):
            img_path = os.path.join(instr_path, f)
        else:
            img_path = os.path.join(instr_path, f + '.png')
        img = np.array(Image.open(img_path)).astype(np.int32)

        cnt = Counter(img.flatten())
        element_counts += cnt
    return OrderedDict(sorted(element_counts.items()))


def ordered_dict_to_list(ordered_dict, num_classes):
    """
    Convert an OrderedDict to a list, filling missing class indices with zero.

    Parameters:
    - ordered_dict (OrderedDict): The input OrderedDict with class counts.
    - num_classes (int): The total number of classes.

    Returns:
    - list: A list of length `num_classes` with counts from the OrderedDict, filling missing indices with zero.
    """
    result_list = [0] * num_classes  # Initialize a list with zeros
    for key, value in ordered_dict.items():
        if key < num_classes:
            result_list[key] = value
    return np.array(result_list).reshape(num_classes, 1)


def calculate_weighted_f1(conf_matrix):
    """
    Calculate the weighted F1 score from a given confusion matrix.

    Parameters:
    - conf_matrix (np.ndarray): 2D array representing the confusion matrix with predicted labels as columns
                                and true labels as rows.

    Returns:
    - float: The weighted F1 score.
    """
    # Extract y_true and y_pred from the confusion matrix
    y_true = []
    y_pred = []
    for true_class in range(conf_matrix.shape[0]):
        for pred_class in range(conf_matrix.shape[1]):
            count = int(round(conf_matrix[true_class, pred_class]))  # Ensure count is an integer
            y_true.extend([true_class] * count)
            y_pred.extend([pred_class] * count)

    # Calculate and return the weighted F1 score
    return f1_score(y_true, y_pred, average='weighted', zero_division=0)

In [3]:
#RFI_complex_a0.5
tmp=stitch_count('../dataset-mit/instruction',['../dataset-mit/val_real.txt','../dataset-mit/val_synt.txt'])
true_cnt=ordered_dict_to_list(tmp,17)
conf=np.array([
[0.937,0.041,0.001,0.003,0.006,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.002,0.001,0.000],
[0.040,0.934,0.001,0.004,0.002,0.000,0.003,0.000,0.006,0.000,0.006,0.000,0.001,0.002,0.000,0.001,0.000],
[0.196,0.064,0.567,0.173,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.220,0.095,0.092,0.570,0.003,0.002,0.009,0.000,0.003,0.000,0.004,0.000,0.000,0.001,0.000,0.000,0.000],
[0.350,0.080,0.000,0.170,0.388,0.000,0.002,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.299,0.004,0.000,0.153,0.011,0.000,0.509,0.000,0.000,0.000,0.024,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.002,0.312,0.000,0.030,0.003,0.000,0.000,0.000,0.616,0.000,0.037,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.002,0.325,0.000,0.046,0.000,0.000,0.030,0.000,0.006,0.000,0.591,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.198,0.216,0.000,0.000,0.000,0.000,0.000,0.000,0.069,0.000,0.001,0.000,0.480,0.006,0.000,0.030,0.000],
[0.291,0.159,0.000,0.018,0.000,0.000,0.000,0.000,0.000,0.000,0.025,0.000,0.015,0.420,0.061,0.003,0.008],
[0.160,0.226,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.037,0.000,0.000,0.142,0.425,0.000,0.000],
[0.147,0.230,0.000,0.000,0.000,0.000,0.000,0.000,0.017,0.000,0.017,0.000,0.104,0.000,0.006,0.478,0.000],
[0.000,0.975,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.025]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.9017381056824617

In [4]:
#RFINet_notran_noaug_newinst
tmp=stitch_count('../dataset-mit/instruction',['../dataset-mit/val_real.txt','../dataset-mit/val_synt.txt'])
true_cnt=ordered_dict_to_list(tmp,14)
conf=np.array([
[0.985,0.007,0.001,0.001,0.001,0.000,0.000,0.002,0.001,0.001,0.001,0.000,0.000,0.000],
[0.045,0.931,0.010,0.000,0.006,0.000,0.000,0.003,0.003,0.001,0.001,0.000,0.000,0.000],
[0.162,0.063,0.742,0.003,0.019,0.000,0.000,0.003,0.007,0.003,0.000,0.000,0.000,0.000],
[0.038,0.001,0.000,0.955,0.000,0.000,0.000,0.003,0.002,0.001,0.000,0.000,0.000,0.000],
[0.041,0.024,0.004,0.011,0.913,0.000,0.000,0.000,0.004,0.002,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.059,0.000,0.000,0.002,0.000,0.000,0.000,0.934,0.005,0.000,0.000,0.000,0.000,0.000],
[0.059,0.001,0.000,0.002,0.000,0.000,0.000,0.015,0.923,0.000,0.000,0.000,0.000,0.000],
[0.160,0.059,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.778,0.003,0.000,0.000,0.000],
[0.184,0.035,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.774,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.9726143710823633

In [5]:
#RFINet_front_xferln_MIL_160k
tmp=stitch_count('../dataset/instruction-front',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,14)
conf=np.array([
[0.898,0.031,0.024,0.009,0.004,0.000,0.000,0.002,0.002,0.000,0.000,0.016,0.012,0.001],
[0.126,0.831,0.027,0.005,0.006,0.000,0.000,0.000,0.001,0.001,0.002,0.000,0.001,0.000],
[0.343,0.038,0.535,0.042,0.018,0.000,0.000,0.003,0.002,0.000,0.000,0.000,0.003,0.014],
[0.321,0.006,0.009,0.630,0.014,0.000,0.000,0.015,0.006,0.000,0.000,0.000,0.000,0.000],
[0.590,0.030,0.044,0.031,0.286,0.000,0.000,0.005,0.013,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.386,0.003,0.000,0.000,0.591,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000],
[0.251,0.006,0.006,0.006,0.037,0.000,0.000,0.681,0.012,0.000,0.000,0.000,0.000,0.000],
[0.348,0.021,0.001,0.002,0.003,0.000,0.000,0.032,0.583,0.000,0.000,0.000,0.009,0.000],
[0.402,0.066,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.531,0.000,0.000,0.000,0.000],
[0.558,0.051,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.048,0.343,0.000,0.000,0.000],
[0.621,0.001,0.001,0.002,0.008,0.000,0.001,0.000,0.000,0.000,0.000,0.359,0.003,0.002],
[0.355,0.064,0.015,0.003,0.038,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.524,0.000],
[0.492,0.140,0.107,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.261]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.8209939923534646

In [6]:
#RFINet_front_xferln_160k
tmp=stitch_count('../dataset/instruction-front',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,14)
conf=np.array([
[0.914,0.029,0.025,0.006,0.004,0.000,0.000,0.001,0.002,0.000,0.000,0.011,0.007,0.000],
[0.141,0.820,0.018,0.004,0.008,0.000,0.000,0.000,0.000,0.001,0.003,0.000,0.005,0.000],
[0.358,0.052,0.508,0.038,0.030,0.000,0.000,0.003,0.001,0.000,0.000,0.000,0.009,0.001],
[0.276,0.009,0.009,0.679,0.008,0.000,0.000,0.004,0.015,0.000,0.000,0.000,0.000,0.000],
[0.546,0.032,0.102,0.007,0.303,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.001,0.000],
[0.667,0.111,0.000,0.000,0.222,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.295,0.004,0.004,0.000,0.486,0.000,0.210,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.286,0.017,0.007,0.022,0.051,0.000,0.000,0.598,0.018,0.000,0.000,0.000,0.000,0.000],
[0.345,0.032,0.012,0.023,0.004,0.000,0.000,0.015,0.558,0.000,0.011,0.000,0.000,0.000],
[0.238,0.178,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.581,0.000,0.000,0.000,0.000],
[0.366,0.052,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.081,0.501,0.000,0.000,0.000],
[0.546,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.453,0.000,0.000],
[0.251,0.042,0.031,0.001,0.044,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.629,0.000],
[0.395,0.386,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.183]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.831402695082505

In [7]:
#RFINet_complete_MIL
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.895,0.043,0.004,0.003,0.000,0.001,0.000,0.010,0.008,0.003,0.003,0.009,0.000,0.000,0.000,0.001,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000],
[0.125,0.588,0.032,0.031,0.004,0.000,0.000,0.000,0.001,0.004,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.185,0.010,0.000,0.000,0.005,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000],
[0.499,0.195,0.163,0.108,0.000,0.000,0.000,0.000,0.000,0.004,0.004,0.000,0.000,0.000,0.012,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000],
[0.543,0.032,0.186,0.163,0.002,0.006,0.000,0.023,0.006,0.000,0.000,0.001,0.000,0.000,0.000,0.006,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.160,0.112,0.143,0.007,0.248,0.046,0.002,0.002,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.076,0.001,0.162,0.004,0.000,0.000,0.004,0.000,0.004,0.000,0.022,0.000,0.000,0.000,0.001,0.000,0.001,0.000,0.000],
[0.459,0.059,0.002,0.005,0.009,0.138,0.009,0.000,0.014,0.000,0.000,0.251,0.000,0.018,0.000,0.020,0.017,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.758,0.000,0.000,0.010,0.069,0.046,0.036,0.004,0.000,0.000,0.000,0.024,0.000,0.000,0.000,0.050,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.776,0.001,0.000,0.015,0.000,0.000,0.007,0.106,0.000,0.000,0.000,0.020,0.002,0.000,0.000,0.005,0.067,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.736,0.049,0.000,0.000,0.004,0.014,0.004,0.000,0.134,0.000,0.000,0.031,0.000,0.000,0.000,0.000,0.028,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.620,0.140,0.000,0.083,0.000,0.000,0.017,0.000,0.000,0.116,0.017,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.702,0.162,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.058,0.071,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.187,0.064,0.005,0.001,0.032,0.226,0.000,0.000,0.001,0.027,0.000,0.155,0.000,0.016,0.000,0.283,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.325,0.000,0.000,0.513,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.162,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.001,0.000,0.000,0.000,0.488,0.000,0.000,0.000,0.000,0.000,0.000,0.504,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.694,0.000,0.000,0.306,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.409,0.034,0.003,0.000,0.018,0.010,0.009,0.004,0.000,0.000,0.000,0.311,0.000,0.000,0.000,0.185,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.647,0.017,0.000,0.000,0.002,0.008,0.002,0.011,0.049,0.000,0.000,0.077,0.000,0.000,0.000,0.004,0.181,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.046,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.801,0.039,0.000,0.000,0.003,0.001,0.001,0.015,0.002,0.008,0.005,0.005,0.009,0.016,0.013,0.033,0.000],
[0.000,0.075,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.363,0.507,0.001,0.000,0.004,0.000,0.000,0.000,0.000,0.024,0.000,0.000,0.000,0.000,0.007,0.017,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.802,0.000,0.094,0.022,0.000,0.028,0.000,0.000,0.000,0.042,0.000,0.000,0.010,0.002,0.000,0.000,0.000],
[0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.642,0.000,0.061,0.247,0.000,0.043,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.003,0.000],
[0.000,0.143,0.000,0.000,0.112,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.606,0.053,0.000,0.003,0.012,0.000,0.000,0.000,0.034,0.034,0.000,0.000,0.001,0.000,0.001,0.000,0.000],
[0.859,0.116,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.001,0.012,0.000,0.005,0.001,0.000,0.001,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.103,0.000,0.000,0.286,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.455,0.098,0.000,0.001,0.003,0.001,0.000,0.000,0.036,0.008,0.000,0.000,0.005,0.000,0.005,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.388,0.002,0.000,0.000,0.000,0.000,0.000,0.608,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.000,0.129,0.000,0.000,0.049,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.504,0.091,0.000,0.000,0.002,0.000,0.014,0.000,0.129,0.000,0.000,0.000,0.010,0.000,0.072,0.000,0.000],
[0.002,0.051,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.355,0.085,0.004,0.001,0.004,0.010,0.000,0.000,0.000,0.381,0.000,0.000,0.100,0.004,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.563,0.000,0.000,0.000,0.000,0.000,0.000,0.074,0.000,0.000,0.301,0.000,0.000,0.062,0.000,0.000,0.000],
[0.000,0.001,0.000,0.000,0.022,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.687,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.001,0.000,0.001,0.284,0.000,0.000,0.001,0.000,0.000],
[0.000,0.019,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.498,0.037,0.002,0.001,0.030,0.000,0.000,0.000,0.000,0.148,0.000,0.000,0.257,0.000,0.007,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.516,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.045,0.000,0.000,0.437,0.000,0.000,0.000],
[0.000,0.139,0.000,0.000,0.027,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.447,0.053,0.000,0.001,0.050,0.000,0.002,0.000,0.003,0.000,0.000,0.000,0.003,0.000,0.275,0.000,0.000],
[0.002,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.758,0.025,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.002,0.195,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.7157356720206056

In [8]:
#RFINet_complete
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.954,0.021,0.004,0.002,0.001,0.002,0.000,0.002,0.003,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.031,0.799,0.004,0.000,0.011,0.000,0.000,0.006,0.000,0.001,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.106,0.016,0.000,0.000,0.004,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.016,0.000,0.000],
[0.454,0.051,0.469,0.005,0.000,0.000,0.000,0.000,0.000,0.005,0.011,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.311,0.157,0.130,0.370,0.000,0.000,0.002,0.017,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.077,0.150,0.145,0.003,0.306,0.011,0.002,0.004,0.000,0.000,0.000,0.077,0.000,0.000,0.000,0.040,0.000,0.127,0.016,0.000,0.000,0.018,0.000,0.017,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.163,0.031,0.005,0.009,0.023,0.672,0.000,0.000,0.007,0.000,0.000,0.062,0.000,0.000,0.000,0.020,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.329,0.000,0.000,0.067,0.000,0.004,0.088,0.000,0.024,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.484,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.118,0.000,0.000,0.029,0.007,0.000,0.000,0.785,0.005,0.000,0.000,0.010,0.000,0.000,0.000,0.012,0.033,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.226,0.037,0.000,0.023,0.020,0.042,0.016,0.016,0.505,0.000,0.000,0.023,0.000,0.000,0.000,0.000,0.091,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.148,0.102,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.746,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.130,0.060,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.027,0.779,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.166,0.048,0.033,0.032,0.010,0.045,0.000,0.000,0.000,0.025,0.002,0.598,0.000,0.010,0.000,0.022,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.504,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.162,0.171,0.000,0.000,0.000,0.162,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.977,0.000,0.000,0.000,0.018,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.833,0.083,0.000,0.028,0.000,0.000,0.000,0.000,0.000,0.056,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.070,0.030,0.005,0.000,0.012,0.014,0.002,0.000,0.000,0.000,0.000,0.051,0.000,0.000,0.000,0.804,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.240,0.027,0.001,0.010,0.001,0.008,0.008,0.050,0.029,0.004,0.000,0.011,0.000,0.000,0.000,0.002,0.609,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.003,0.029,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.839,0.030,0.000,0.000,0.004,0.000,0.003,0.011,0.006,0.005,0.010,0.003,0.007,0.020,0.009,0.015,0.000],
[0.000,0.068,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.322,0.558,0.000,0.000,0.003,0.000,0.001,0.000,0.000,0.030,0.000,0.000,0.000,0.000,0.014,0.002,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.634,0.016,0.148,0.050,0.008,0.043,0.000,0.000,0.000,0.070,0.000,0.000,0.028,0.000,0.003,0.000,0.000],
[0.000,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.373,0.000,0.044,0.251,0.000,0.233,0.000,0.000,0.000,0.000,0.000,0.000,0.084,0.000,0.000,0.000,0.000],
[0.000,0.086,0.000,0.000,0.082,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.718,0.006,0.010,0.000,0.090,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000],
[0.003,0.856,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.120,0.013,0.000,0.001,0.000,0.000,0.004,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000],
[0.000,0.119,0.000,0.000,0.227,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.393,0.154,0.000,0.000,0.018,0.000,0.050,0.000,0.023,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.204,0.001,0.000,0.000,0.000,0.000,0.000,0.795,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.000,0.116,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.478,0.070,0.000,0.000,0.006,0.000,0.001,0.000,0.309,0.002,0.000,0.000,0.002,0.000,0.001,0.000,0.000],
[0.039,0.150,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.171,0.078,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.394,0.000,0.000,0.168,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.383,0.000,0.000,0.000,0.000,0.000,0.000,0.033,0.000,0.000,0.467,0.000,0.000,0.117,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.775,0.001,0.000,0.000,0.001,0.000,0.003,0.000,0.010,0.000,0.001,0.200,0.000,0.000,0.000,0.000,0.000],
[0.000,0.025,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.315,0.008,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.211,0.000,0.000,0.436,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.501,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.032,0.000,0.000,0.467,0.000,0.000,0.000],
[0.000,0.042,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.444,0.033,0.000,0.000,0.063,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.006,0.000,0.393,0.000,0.000],
[0.023,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.521,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.029,0.000,0.000,0.000,0.000,0.000,0.409,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.8079517598466229

In [9]:
#xfer_complete_frompred_MIL
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.775,0.069,0.000,0.006,0.005,0.009,0.000,0.006,0.003,0.004,0.006,0.009,0.000,0.000,0.000,0.008,0.018,0.077,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.391,0.447,0.003,0.009,0.005,0.008,0.000,0.007,0.001,0.012,0.005,0.004,0.000,0.001,0.000,0.002,0.007,0.074,0.016,0.000,0.000,0.000,0.001,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.371,0.396,0.038,0.007,0.108,0.005,0.000,0.001,0.000,0.002,0.000,0.039,0.000,0.006,0.000,0.000,0.006,0.014,0.003,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.298,0.099,0.001,0.129,0.124,0.025,0.002,0.004,0.001,0.002,0.049,0.017,0.000,0.000,0.000,0.006,0.014,0.163,0.006,0.000,0.000,0.000,0.002,0.000,0.043,0.000,0.000,0.000,0.000,0.000,0.016,0.000,0.000,0.000],
[0.334,0.089,0.002,0.055,0.204,0.021,0.004,0.001,0.001,0.005,0.001,0.028,0.000,0.001,0.000,0.029,0.013,0.154,0.014,0.000,0.000,0.000,0.004,0.001,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.707,0.055,0.003,0.005,0.047,0.043,0.000,0.001,0.000,0.001,0.001,0.013,0.000,0.000,0.000,0.061,0.029,0.032,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.316,0.018,0.000,0.038,0.339,0.011,0.000,0.005,0.000,0.000,0.000,0.213,0.000,0.000,0.000,0.000,0.010,0.035,0.000,0.000,0.000,0.000,0.004,0.000,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.743,0.122,0.000,0.002,0.003,0.004,0.000,0.018,0.006,0.000,0.000,0.026,0.000,0.002,0.000,0.004,0.003,0.059,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.607,0.118,0.002,0.001,0.032,0.008,0.000,0.009,0.007,0.001,0.000,0.101,0.000,0.000,0.000,0.002,0.004,0.067,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.039,0.000,0.000],
[0.873,0.063,0.000,0.000,0.001,0.003,0.000,0.000,0.001,0.004,0.005,0.008,0.000,0.001,0.000,0.001,0.003,0.038,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.521,0.407,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.021,0.004,0.009,0.000,0.000,0.000,0.000,0.001,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.470,0.054,0.000,0.004,0.013,0.078,0.001,0.031,0.004,0.002,0.006,0.042,0.000,0.000,0.000,0.127,0.115,0.047,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.451,0.003,0.000,0.055,0.315,0.000,0.000,0.000,0.000,0.000,0.000,0.082,0.000,0.000,0.000,0.000,0.034,0.053,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.371,0.293,0.030,0.001,0.000,0.043,0.000,0.059,0.041,0.044,0.001,0.006,0.000,0.007,0.000,0.020,0.035,0.039,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.463,0.134,0.000,0.057,0.007,0.000,0.000,0.000,0.000,0.002,0.000,0.026,0.000,0.000,0.000,0.060,0.000,0.251,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.734,0.035,0.000,0.002,0.019,0.046,0.000,0.002,0.002,0.000,0.000,0.020,0.000,0.000,0.000,0.083,0.020,0.029,0.000,0.000,0.000,0.000,0.002,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.384,0.028,0.000,0.003,0.044,0.016,0.000,0.001,0.001,0.002,0.000,0.341,0.000,0.008,0.000,0.018,0.111,0.032,0.001,0.000,0.000,0.000,0.003,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.594,0.059,0.000,0.008,0.009,0.001,0.000,0.000,0.000,0.001,0.002,0.001,0.000,0.000,0.000,0.001,0.003,0.232,0.041,0.000,0.000,0.000,0.000,0.000,0.020,0.000,0.001,0.005,0.001,0.001,0.000,0.019,0.001,0.000],
[0.162,0.085,0.002,0.010,0.011,0.001,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.285,0.424,0.000,0.000,0.000,0.000,0.000,0.012,0.000,0.001,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.266,0.037,0.001,0.077,0.025,0.001,0.000,0.000,0.000,0.000,0.000,0.023,0.000,0.000,0.000,0.013,0.004,0.171,0.148,0.000,0.000,0.000,0.001,0.000,0.233,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.086,0.168,0.000,0.091,0.164,0.000,0.001,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.001,0.221,0.135,0.000,0.000,0.000,0.001,0.000,0.130,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.272,0.060,0.003,0.086,0.090,0.001,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.004,0.001,0.329,0.040,0.000,0.000,0.000,0.000,0.000,0.098,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000],
[0.266,0.260,0.000,0.027,0.054,0.003,0.000,0.000,0.000,0.001,0.000,0.131,0.000,0.088,0.000,0.000,0.015,0.050,0.049,0.000,0.000,0.000,0.006,0.000,0.046,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.250,0.132,0.000,0.090,0.066,0.032,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.009,0.002,0.298,0.028,0.000,0.000,0.000,0.001,0.026,0.056,0.001,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000],
[0.039,0.009,0.000,0.008,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.367,0.015,0.000,0.000,0.000,0.000,0.000,0.540,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000],
[0.209,0.028,0.003,0.204,0.157,0.015,0.000,0.000,0.000,0.001,0.005,0.014,0.000,0.000,0.000,0.040,0.004,0.153,0.017,0.000,0.000,0.000,0.000,0.038,0.104,0.001,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000],
[0.086,0.255,0.000,0.001,0.009,0.002,0.000,0.000,0.000,0.001,0.002,0.001,0.000,0.000,0.000,0.000,0.002,0.338,0.129,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.034,0.000,0.002,0.019,0.000,0.090,0.020,0.000],
[0.090,0.014,0.000,0.000,0.001,0.001,0.000,0.000,0.001,0.004,0.021,0.003,0.000,0.000,0.000,0.000,0.002,0.582,0.000,0.000,0.000,0.000,0.000,0.001,0.003,0.000,0.001,0.139,0.023,0.000,0.009,0.106,0.000,0.000],
[0.218,0.009,0.000,0.000,0.001,0.007,0.000,0.001,0.000,0.009,0.023,0.006,0.000,0.000,0.000,0.000,0.008,0.575,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.053,0.017,0.001,0.003,0.068,0.000,0.000],
[0.061,0.239,0.000,0.000,0.008,0.002,0.000,0.000,0.000,0.001,0.003,0.001,0.000,0.001,0.000,0.000,0.002,0.381,0.106,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.031,0.001,0.001,0.058,0.000,0.076,0.026,0.000],
[0.247,0.346,0.001,0.001,0.000,0.002,0.000,0.000,0.002,0.002,0.005,0.003,0.000,0.000,0.000,0.000,0.001,0.307,0.003,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.001,0.024,0.008,0.000,0.001,0.045,0.000,0.000],
[0.029,0.033,0.000,0.006,0.003,0.001,0.000,0.001,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.352,0.002,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.007,0.001,0.000,0.001,0.000,0.553,0.001,0.000],
[0.189,0.259,0.000,0.002,0.005,0.003,0.000,0.000,0.000,0.001,0.002,0.002,0.000,0.000,0.000,0.000,0.002,0.325,0.112,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.011,0.000,0.001,0.044,0.000,0.018,0.014,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.3938373067223805

In [10]:
#xfer_complete_frompred
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.946,0.007,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.007,0.029,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.091,0.843,0.001,0.003,0.002,0.002,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.001,0.001,0.035,0.014,0.000,0.000,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.113,0.175,0.318,0.014,0.072,0.001,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.007,0.001,0.108,0.173,0.000,0.000,0.000,0.008,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.102,0.020,0.006,0.537,0.205,0.007,0.001,0.002,0.000,0.000,0.001,0.005,0.000,0.000,0.000,0.014,0.004,0.037,0.010,0.000,0.000,0.000,0.001,0.000,0.048,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.107,0.044,0.002,0.101,0.569,0.005,0.003,0.000,0.000,0.001,0.006,0.017,0.000,0.000,0.000,0.007,0.004,0.059,0.017,0.000,0.000,0.000,0.001,0.000,0.055,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.069,0.007,0.001,0.003,0.003,0.725,0.000,0.042,0.001,0.000,0.011,0.015,0.000,0.000,0.000,0.076,0.006,0.027,0.001,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000],
[0.095,0.011,0.000,0.031,0.381,0.001,0.334,0.013,0.001,0.000,0.000,0.008,0.000,0.000,0.000,0.016,0.011,0.016,0.001,0.000,0.000,0.000,0.053,0.000,0.027,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.067,0.010,0.000,0.006,0.008,0.027,0.000,0.702,0.024,0.002,0.002,0.003,0.000,0.000,0.000,0.053,0.011,0.057,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.002,0.000,0.001,0.000,0.001,0.023,0.000,0.000],
[0.068,0.010,0.000,0.010,0.002,0.047,0.000,0.107,0.411,0.073,0.036,0.005,0.000,0.000,0.000,0.068,0.014,0.072,0.001,0.000,0.000,0.000,0.012,0.000,0.000,0.001,0.004,0.001,0.001,0.000,0.001,0.055,0.000,0.000],
[0.021,0.011,0.000,0.000,0.001,0.015,0.000,0.003,0.030,0.837,0.033,0.001,0.000,0.000,0.000,0.004,0.002,0.012,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.024,0.003,0.000,0.001,0.000,0.000,0.000],
[0.022,0.020,0.000,0.000,0.001,0.019,0.000,0.001,0.016,0.026,0.858,0.001,0.000,0.000,0.000,0.000,0.002,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.014,0.000,0.000,0.000,0.000,0.000],
[0.196,0.020,0.003,0.001,0.017,0.015,0.004,0.002,0.000,0.000,0.000,0.647,0.000,0.002,0.000,0.016,0.020,0.032,0.023,0.000,0.000,0.000,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.229,0.066,0.000,0.029,0.269,0.000,0.030,0.071,0.000,0.000,0.000,0.064,0.208,0.000,0.000,0.000,0.023,0.000,0.000,0.000,0.000,0.000,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.208,0.296,0.000,0.000,0.021,0.013,0.000,0.000,0.000,0.000,0.000,0.141,0.000,0.310,0.000,0.000,0.000,0.001,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.311,0.120,0.022,0.369,0.031,0.000,0.018,0.002,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.003,0.000,0.101,0.015,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.066,0.008,0.000,0.008,0.014,0.089,0.000,0.012,0.016,0.004,0.000,0.014,0.000,0.000,0.000,0.732,0.004,0.019,0.001,0.000,0.000,0.000,0.002,0.001,0.002,0.001,0.001,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.314,0.008,0.000,0.001,0.004,0.005,0.003,0.006,0.002,0.001,0.000,0.036,0.000,0.000,0.000,0.005,0.573,0.039,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.650,0.045,0.000,0.003,0.002,0.001,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.248,0.025,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.004,0.001,0.000,0.000,0.006,0.000,0.000],
[0.139,0.087,0.003,0.004,0.005,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.188,0.562,0.000,0.000,0.000,0.001,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.001,0.000],
[0.184,0.115,0.009,0.024,0.140,0.000,0.000,0.008,0.000,0.000,0.000,0.041,0.000,0.000,0.000,0.009,0.000,0.257,0.072,0.000,0.000,0.000,0.001,0.000,0.140,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.045,0.194,0.000,0.084,0.365,0.001,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.002,0.000,0.166,0.066,0.000,0.000,0.000,0.001,0.000,0.075,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.175,0.085,0.002,0.229,0.190,0.001,0.001,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.008,0.001,0.212,0.013,0.000,0.000,0.000,0.000,0.000,0.062,0.000,0.002,0.000,0.001,0.000,0.000,0.007,0.001,0.000],
[0.042,0.518,0.000,0.061,0.008,0.007,0.000,0.014,0.003,0.001,0.000,0.001,0.000,0.003,0.000,0.014,0.058,0.040,0.034,0.000,0.000,0.000,0.179,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.143,0.066,0.000,0.300,0.056,0.041,0.000,0.005,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.036,0.002,0.188,0.019,0.000,0.000,0.000,0.000,0.070,0.063,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.043,0.007,0.001,0.007,0.040,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.163,0.040,0.000,0.000,0.000,0.000,0.000,0.686,0.000,0.002,0.000,0.000,0.000,0.000,0.008,0.000,0.000],
[0.250,0.051,0.001,0.202,0.027,0.076,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.035,0.002,0.219,0.027,0.000,0.000,0.002,0.000,0.000,0.040,0.066,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.065,0.208,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.298,0.128,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.042,0.000,0.002,0.056,0.000,0.142,0.044,0.000],
[0.082,0.002,0.000,0.000,0.000,0.005,0.000,0.000,0.002,0.021,0.023,0.001,0.000,0.000,0.000,0.000,0.006,0.491,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.255,0.047,0.000,0.001,0.063,0.001,0.000],
[0.183,0.002,0.000,0.000,0.000,0.008,0.000,0.000,0.005,0.027,0.030,0.004,0.000,0.000,0.000,0.000,0.006,0.514,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.117,0.058,0.000,0.000,0.042,0.000,0.000],
[0.070,0.287,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.277,0.069,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.034,0.004,0.001,0.081,0.000,0.085,0.083,0.000],
[0.207,0.269,0.000,0.001,0.000,0.006,0.000,0.000,0.004,0.021,0.017,0.003,0.000,0.000,0.000,0.000,0.007,0.271,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.116,0.032,0.000,0.001,0.043,0.000,0.000],
[0.031,0.028,0.000,0.003,0.003,0.004,0.000,0.000,0.000,0.002,0.005,0.000,0.000,0.000,0.000,0.000,0.001,0.141,0.002,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.008,0.005,0.007,0.003,0.000,0.749,0.005,0.000],
[0.105,0.325,0.000,0.001,0.001,0.006,0.000,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.000,0.002,0.232,0.217,0.000,0.000,0.000,0.003,0.000,0.003,0.000,0.010,0.000,0.000,0.038,0.000,0.014,0.041,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.5272412783480616

In [11]:
#xfer_complete_fromtrue_MIL
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.771,0.069,0.001,0.006,0.005,0.008,0.000,0.007,0.003,0.006,0.005,0.010,0.000,0.000,0.000,0.010,0.017,0.079,0.002,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.423,0.429,0.004,0.018,0.009,0.007,0.000,0.008,0.003,0.005,0.014,0.004,0.000,0.001,0.000,0.005,0.006,0.048,0.014,0.000,0.000,0.000,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.301,0.368,0.040,0.032,0.112,0.000,0.000,0.003,0.000,0.000,0.007,0.091,0.000,0.009,0.000,0.003,0.004,0.019,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.318,0.113,0.000,0.130,0.107,0.025,0.003,0.007,0.006,0.016,0.041,0.007,0.000,0.000,0.000,0.011,0.002,0.184,0.004,0.000,0.000,0.000,0.004,0.000,0.020,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000],
[0.294,0.123,0.012,0.032,0.228,0.038,0.003,0.004,0.000,0.000,0.001,0.024,0.000,0.000,0.000,0.025,0.000,0.119,0.023,0.000,0.000,0.001,0.003,0.000,0.040,0.024,0.001,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.720,0.058,0.000,0.003,0.009,0.070,0.000,0.000,0.000,0.000,0.000,0.014,0.000,0.000,0.000,0.058,0.022,0.039,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000],
[0.146,0.029,0.000,0.025,0.377,0.005,0.011,0.000,0.000,0.000,0.000,0.359,0.000,0.000,0.000,0.000,0.001,0.016,0.000,0.000,0.000,0.011,0.005,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.669,0.126,0.000,0.001,0.023,0.000,0.000,0.007,0.015,0.000,0.000,0.095,0.000,0.000,0.000,0.000,0.004,0.030,0.000,0.000,0.000,0.001,0.007,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.020,0.000,0.000],
[0.743,0.122,0.000,0.002,0.002,0.001,0.000,0.000,0.020,0.000,0.000,0.012,0.000,0.000,0.000,0.000,0.014,0.057,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.023,0.000,0.000],
[0.469,0.416,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.017,0.010,0.000,0.000,0.000,0.005,0.000,0.077,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.812,0.093,0.000,0.001,0.003,0.000,0.000,0.000,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.086,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.465,0.053,0.001,0.005,0.018,0.114,0.002,0.038,0.011,0.010,0.002,0.051,0.000,0.000,0.000,0.071,0.130,0.017,0.001,0.000,0.000,0.000,0.007,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.533,0.029,0.000,0.004,0.380,0.000,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.047,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.393,0.330,0.000,0.008,0.000,0.047,0.000,0.036,0.035,0.056,0.005,0.003,0.000,0.005,0.000,0.017,0.025,0.002,0.000,0.000,0.000,0.000,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.256,0.299,0.000,0.210,0.001,0.000,0.000,0.065,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.166,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.707,0.058,0.009,0.011,0.039,0.078,0.000,0.000,0.000,0.000,0.003,0.005,0.000,0.000,0.000,0.056,0.016,0.018,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.368,0.033,0.000,0.001,0.039,0.017,0.001,0.001,0.000,0.001,0.002,0.367,0.000,0.006,0.000,0.015,0.128,0.012,0.000,0.000,0.000,0.000,0.002,0.004,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000],
[0.564,0.050,0.000,0.006,0.007,0.000,0.000,0.001,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.001,0.002,0.252,0.042,0.000,0.000,0.001,0.000,0.001,0.022,0.000,0.001,0.015,0.002,0.002,0.000,0.021,0.004,0.000],
[0.148,0.064,0.006,0.003,0.010,0.001,0.000,0.001,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.282,0.450,0.000,0.000,0.000,0.002,0.001,0.024,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.001,0.000],
[0.397,0.074,0.000,0.110,0.035,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.174,0.149,0.000,0.000,0.000,0.000,0.000,0.061,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.083,0.122,0.000,0.027,0.174,0.000,0.000,0.000,0.002,0.000,0.000,0.011,0.000,0.000,0.000,0.001,0.001,0.292,0.099,0.003,0.004,0.001,0.000,0.000,0.147,0.000,0.000,0.000,0.000,0.000,0.000,0.034,0.000,0.000],
[0.233,0.046,0.003,0.033,0.082,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.011,0.002,0.385,0.022,0.000,0.000,0.001,0.001,0.000,0.141,0.000,0.001,0.000,0.000,0.000,0.000,0.033,0.000,0.000],
[0.332,0.215,0.000,0.023,0.018,0.014,0.000,0.000,0.000,0.000,0.005,0.118,0.000,0.073,0.000,0.000,0.024,0.089,0.028,0.000,0.000,0.000,0.049,0.003,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.077,0.006,0.000,0.004,0.752,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.015,0.037,0.000,0.000,0.000,0.030,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.048,0.000,0.000],
[0.018,0.005,0.000,0.001,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.367,0.017,0.000,0.000,0.000,0.000,0.000,0.571,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.139,0.090,0.000,0.030,0.021,0.089,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.002,0.017,0.580,0.024,0.000,0.000,0.000,0.001,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000],
[0.055,0.271,0.002,0.000,0.001,0.004,0.000,0.000,0.002,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.004,0.314,0.131,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.024,0.003,0.004,0.009,0.000,0.160,0.012,0.000],
[0.043,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.686,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.063,0.185,0.000,0.000,0.023,0.000,0.000],
[0.067,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.675,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.107,0.128,0.000,0.000,0.022,0.000,0.000],
[0.051,0.312,0.000,0.000,0.000,0.003,0.000,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.336,0.085,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.023,0.000,0.096,0.086,0.000],
[0.230,0.382,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.305,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.031,0.045,0.000,0.000,0.007,0.000,0.000],
[0.005,0.001,0.000,0.001,0.002,0.002,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.336,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.005,0.000,0.002,0.000,0.000,0.632,0.005,0.000],
[0.069,0.215,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.009,0.305,0.173,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.007,0.004,0.007,0.019,0.000,0.085,0.096,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.40799612378873057

In [13]:
#xfer_complete_fromtrue
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.960,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.006,0.030,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.057,0.913,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.001,0.000,0.000,0.000,0.014,0.008,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.015,0.043,0.500,0.000,0.023,0.000,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.005,0.403,0.000,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.001,0.000,0.002,0.705,0.155,0.000,0.006,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.078,0.000,0.004,0.001,0.000,0.000,0.001,0.011,0.000,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.013,0.015,0.001,0.091,0.725,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.029,0.000,0.041,0.002,0.000,0.000,0.000,0.000,0.000,0.073,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.001,0.000,0.852,0.000,0.034,0.002,0.000,0.031,0.000,0.000,0.000,0.000,0.062,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.008,0.001,0.000,0.000,0.000,0.000,0.007,0.000,0.000],
[0.000,0.000,0.000,0.077,0.408,0.000,0.472,0.005,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.004,0.001,0.036,0.000,0.861,0.045,0.013,0.010,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.004,0.000,0.010,0.001,0.000,0.003,0.000,0.001,0.007,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.086,0.000,0.278,0.338,0.132,0.023,0.000,0.000,0.000,0.000,0.078,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.002,0.012,0.003,0.001,0.042,0.002,0.000],
[0.000,0.000,0.000,0.000,0.001,0.010,0.000,0.003,0.041,0.865,0.021,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.024,0.028,0.002,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.020,0.000,0.001,0.004,0.027,0.894,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.027,0.023,0.000,0.000,0.000,0.000,0.000],
[0.158,0.002,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.758,0.000,0.000,0.000,0.000,0.016,0.033,0.023,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.041,0.276,0.000,0.128,0.304,0.000,0.000,0.000,0.000,0.168,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.080,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.016,0.384,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.064,0.001,0.522,0.000,0.000,0.005,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.585,0.000,0.000,0.000,0.059,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.353,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.001,0.000,0.056,0.000,0.012,0.022,0.006,0.001,0.000,0.000,0.000,0.000,0.889,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.007,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000],
[0.261,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.046,0.000,0.000,0.000,0.000,0.667,0.019,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.613,0.086,0.000,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.271,0.013,0.000,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.116,0.003,0.007,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.048,0.819,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.230,0.311,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.045,0.000,0.001,0.000,0.000,0.000,0.160,0.146,0.096,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.013,0.237,0.000,0.214,0.412,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.073,0.000,0.000,0.000,0.000,0.000,0.000,0.051,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.142,0.000,0.002,0.354,0.198,0.000,0.000,0.036,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.026,0.000,0.130,0.000,0.000,0.000,0.002,0.008,0.000,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.554,0.000,0.046,0.000,0.000,0.000,0.056,0.003,0.000,0.000,0.001,0.000,0.000,0.000,0.022,0.001,0.011,0.051,0.000,0.000,0.000,0.244,0.000,0.000,0.001,0.006,0.000,0.001,0.000,0.000,0.000,0.002,0.000],
[0.000,0.088,0.000,0.066,0.020,0.000,0.000,0.032,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.044,0.000,0.008,0.039,0.000,0.000,0.000,0.004,0.675,0.001,0.022,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.004,0.008,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.005,0.000,0.000,0.000,0.002,0.000,0.962,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.030,0.002,0.000,0.000,0.006,0.113,0.000,0.010,0.019,0.000,0.000,0.002,0.000,0.000,0.000,0.050,0.000,0.003,0.147,0.000,0.000,0.000,0.005,0.008,0.010,0.597,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.198,0.000,0.004,0.002,0.046,0.001,0.000,0.018,0.000,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.298,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.051,0.001,0.008,0.079,0.000,0.214,0.053,0.000],
[0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.003,0.024,0.031,0.045,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.534,0.330,0.001,0.011,0.012,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.001,0.002,0.024,0.027,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.299,0.578,0.002,0.004,0.057,0.000,0.000],
[0.000,0.182,0.000,0.004,0.006,0.054,0.002,0.000,0.016,0.007,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.312,0.000,0.000,0.000,0.000,0.000,0.000,0.011,0.028,0.006,0.009,0.149,0.000,0.131,0.069,0.000],
[0.370,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.006,0.022,0.025,0.000,0.000,0.000,0.000,0.000,0.000,0.166,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.220,0.173,0.001,0.004,0.007,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.003,0.002,0.004,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.004,0.016,0.001,0.011,0.014,0.000,0.932,0.007,0.000],
[0.000,0.112,0.000,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.483,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.052,0.002,0.007,0.016,0.000,0.251,0.072,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.5815922569489625

In [18]:
#xfer_complete_frompred_sj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_sj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.983,0.005,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.001,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.051,0.937,0.001,0.001,0.001,0.001,0.000,0.001,0.000,0.001,0.001,0.000,0.000,0.001,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.108,0.087,0.747,0.017,0.019,0.001,0.001,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.083,0.011,0.005,0.780,0.097,0.003,0.004,0.001,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.005,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.053,0.009,0.006,0.084,0.813,0.006,0.010,0.000,0.000,0.000,0.002,0.012,0.000,0.000,0.000,0.003,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.070,0.006,0.002,0.004,0.002,0.829,0.000,0.032,0.002,0.000,0.016,0.009,0.000,0.000,0.000,0.021,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.086,0.007,0.012,0.194,0.302,0.000,0.352,0.004,0.000,0.000,0.000,0.011,0.003,0.000,0.000,0.010,0.014,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.041,0.004,0.000,0.009,0.000,0.026,0.000,0.804,0.076,0.009,0.000,0.002,0.000,0.000,0.000,0.017,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.050,0.007,0.000,0.001,0.000,0.035,0.000,0.063,0.669,0.100,0.012,0.002,0.000,0.000,0.000,0.045,0.013,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.021,0.011,0.000,0.000,0.001,0.012,0.000,0.003,0.039,0.893,0.013,0.001,0.000,0.000,0.000,0.001,0.003,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.026,0.009,0.000,0.000,0.000,0.018,0.000,0.000,0.013,0.010,0.915,0.001,0.000,0.001,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.211,0.005,0.003,0.004,0.006,0.015,0.000,0.002,0.000,0.000,0.000,0.720,0.000,0.003,0.000,0.011,0.019,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.112,0.003,0.014,0.359,0.034,0.000,0.067,0.058,0.000,0.000,0.000,0.052,0.251,0.000,0.000,0.036,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.157,0.357,0.000,0.001,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.110,0.000,0.360,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.221,0.014,0.025,0.722,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.057,0.005,0.002,0.012,0.003,0.029,0.000,0.010,0.019,0.002,0.000,0.008,0.000,0.003,0.000,0.846,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.202,0.005,0.001,0.002,0.000,0.005,0.001,0.011,0.002,0.001,0.000,0.022,0.000,0.000,0.000,0.007,0.736,0.000,0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.042,0.478,0.000,0.001,0.000,0.006,0.000,0.002,0.009,0.000,0.000,0.008,0.000,0.015,0.000,0.000,0.033,0.000,0.000,0.000,0.000,0.000,0.404,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.950077597309929

In [19]:
#xfer_complete_frompred_mj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_mj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.547,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.374,0.063,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.001,0.000,0.000,0.001,0.000,0.004,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.020,0.000,0.000,0.287,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.409,0.044,0.000,0.000,0.000,0.000,0.000,0.237,0.000,0.000,0.000,0.000,0.001,0.000,0.002,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.030,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.912,0.033,0.000,0.000,0.000,0.000,0.000,0.011,0.000,0.001,0.003,0.001,0.000,0.000,0.006,0.001,0.000],
[0.000,0.073,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.347,0.567,0.000,0.000,0.000,0.000,0.000,0.006,0.000,0.001,0.000,0.000,0.001,0.000,0.002,0.001,0.000],
[0.000,0.021,0.000,0.000,0.020,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.592,0.209,0.000,0.000,0.000,0.000,0.000,0.154,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.002,0.000],
[0.000,0.060,0.000,0.000,0.153,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.396,0.218,0.000,0.000,0.000,0.000,0.000,0.172,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.033,0.000,0.000,0.172,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.538,0.059,0.000,0.000,0.000,0.000,0.001,0.189,0.000,0.001,0.000,0.000,0.000,0.000,0.005,0.001,0.000],
[0.000,0.045,0.000,0.000,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.553,0.248,0.000,0.000,0.000,0.000,0.000,0.121,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000],
[0.000,0.005,0.000,0.000,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.530,0.049,0.000,0.000,0.000,0.000,0.029,0.350,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.205,0.081,0.000,0.000,0.000,0.000,0.000,0.705,0.000,0.001,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.000,0.013,0.000,0.000,0.023,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.757,0.061,0.000,0.000,0.000,0.000,0.019,0.119,0.005,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000],
[0.000,0.157,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.457,0.119,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.073,0.002,0.001,0.031,0.000,0.131,0.028,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.662,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.240,0.026,0.000,0.000,0.071,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.734,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.184,0.027,0.001,0.000,0.051,0.000,0.000],
[0.000,0.170,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.393,0.121,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.061,0.002,0.002,0.102,0.000,0.060,0.089,0.000],
[0.000,0.199,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.575,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.128,0.016,0.000,0.000,0.079,0.000,0.000],
[0.000,0.015,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.210,0.007,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.005,0.003,0.002,0.002,0.000,0.747,0.000,0.000],
[0.000,0.150,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.405,0.172,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.042,0.000,0.000,0.079,0.000,0.019,0.130,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.7395524148928677

In [14]:
#xfer_complete_fromtrue_sj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_sj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.995,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.009,0.989,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.008,0.028,0.919,0.000,0.044,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.001,0.000,0.000,0.888,0.083,0.000,0.012,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.009,0.000,0.000,0.069,0.905,0.000,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.982,0.000,0.008,0.005,0.000,0.003,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.056,0.255,0.000,0.660,0.003,0.000,0.000,0.000,0.000,0.023,0.000,0.000,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.007,0.000,0.013,0.000,0.931,0.023,0.002,0.000,0.000,0.000,0.000,0.000,0.024,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.002,0.000,0.039,0.000,0.033,0.830,0.064,0.000,0.000,0.000,0.000,0.000,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.011,0.000,0.000,0.024,0.959,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.009,0.981,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.089,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.899,0.000,0.000,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.233,0.146,0.000,0.227,0.000,0.000,0.000,0.000,0.000,0.393,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.014,0.398,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.055,0.000,0.508,0.000,0.000,0.022,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.751,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.248,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.002,0.000,0.007,0.000,0.010,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.979,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.083,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.020,0.000,0.000,0.000,0.000,0.897,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.602,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.392,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.9842801406326223

In [17]:
#xfer_complete_fromtrue_mj
tmp=stitch_count('../dataset/instruction-complete',['../dataset/val_real_mj.txt'])
true_cnt=ordered_dict_to_list(tmp,34)
conf=np.array([
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.849,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.134,0.012,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.017,0.000,0.000,0.751,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.185,0.001,0.010,0.003,0.010,0.001,0.000,0.022,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.038,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.935,0.018,0.000,0.000,0.001,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000],
[0.000,0.003,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.157,0.836,0.000,0.000,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.228,0.121,0.641,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.405,0.000,0.000,0.248,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.090,0.002,0.018,0.203,0.024,0.000,0.000,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.340,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.265,0.005,0.000,0.007,0.196,0.026,0.009,0.130,0.020,0.000,0.000,0.000,0.002,0.000,0.000,0.000,0.000],
[0.000,0.079,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.018,0.344,0.000,0.000,0.003,0.549,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,0.000,0.000],
[0.000,0.030,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.097,0.008,0.022,0.025,0.022,0.752,0.000,0.022,0.000,0.000,0.000,0.012,0.000,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.038,0.003,0.000,0.001,0.001,0.000,0.000,0.953,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000],
[0.000,0.002,0.000,0.000,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.023,0.103,0.000,0.000,0.006,0.033,0.003,0.012,0.810,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000],
[0.000,0.220,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.030,0.307,0.000,0.000,0.002,0.002,0.005,0.000,0.008,0.089,0.006,0.004,0.101,0.000,0.184,0.043,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.662,0.331,0.002,0.003,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.388,0.555,0.001,0.002,0.050,0.000,0.000],
[0.000,0.251,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.049,0.236,0.000,0.000,0.008,0.003,0.003,0.000,0.007,0.069,0.003,0.001,0.201,0.000,0.087,0.080,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.122,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.329,0.150,0.001,0.397,0.000,0.000,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,0.018,0.002,0.008,0.005,0.000,0.954,0.011,0.000],
[0.000,0.185,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.047,0.346,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.050,0.005,0.006,0.085,0.000,0.149,0.125,0.000],
[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000]])

true_conf=conf*true_cnt
calculate_weighted_f1(true_conf)

0.8627640102442722